In [1]:
import os
import re

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
data_path = 'C:/Users/bitcamp/Desktop/NLP/NLP/tensorflow-ml-nlp-tf2/4.TEXT_CLASSIFICATION/data_in/'
train_clean_data = 'train_clean.csv'

In [3]:
train_data = pd.read_csv(data_path + train_clean_data)

In [4]:
print(train_data.shape)

(25000, 2)


In [5]:
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

In [6]:
sentences = []
for review in reviews:
    sentences.append(review.split())
print(sentences[0])

['stuff', 'going', 'moment', 'mj', 'started', 'listening', 'music', 'watching', 'odd', 'documentary', 'watched', 'wiz', 'watched', 'moonwalker', 'maybe', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'really', 'cool', 'eighties', 'maybe', 'make', 'mind', 'whether', 'guilty', 'innocent', 'moonwalker', 'part', 'biography', 'part', 'feature', 'film', 'remember', 'going', 'see', 'cinema', 'originally', 'released', 'subtle', 'messages', 'mj', 'feeling', 'towards', 'press', 'also', 'obvious', 'message', 'drugs', 'bad', 'kay', 'visually', 'impressive', 'course', 'michael', 'jackson', 'unless', 'remotely', 'like', 'mj', 'anyway', 'going', 'hate', 'find', 'boring', 'may', 'call', 'mj', 'egotist', 'consenting', 'making', 'movie', 'mj', 'fans', 'would', 'say', 'made', 'fans', 'true', 'really', 'nice', 'actual', 'feature', 'film', 'bit', 'finally', 'starts', 'minutes', 'excluding', 'smooth', 'criminal', 'sequence', 'joe', 'pesci', 'convincing', 'psychopathic', 'powerful', 'drug', 'lord', 

##### num_features = 300    워드 백터 특징값 수
##### min_word_count = 40   단어에 대한 최소 빈도 수
##### num_workers = 4       프로세스 개수
##### context = 10          컨텍스트 윈도 개수
##### downsampling = 1e-3    다운 샘플링 비율

In [7]:
num_features = 300    
min_word_count = 40   
num_workers = 4       
context = 10          
downsampling = 1e-3 

In [8]:
import logging
logging.basicConfig(format='%(asctime)s: %(levelname)s : %(message)s', level=logging.INFO)

In [10]:
from gensim.models import word2vec

model = word2vec.Word2Vec(sentences, workers=num_workers, \
           size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

2020-10-26 22:41:47,383: INFO : collecting all words and their counts
2020-10-26 22:41:47,384: INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-10-26 22:41:47,544: INFO : PROGRESS: at sentence #10000, processed 1205223 words, keeping 51374 word types
2020-10-26 22:41:47,711: INFO : PROGRESS: at sentence #20000, processed 2396605 words, keeping 67660 word types
2020-10-26 22:41:47,798: INFO : collected 74065 word types from a corpus of 2988089 raw words and 25000 sentences
2020-10-26 22:41:47,799: INFO : Loading a fresh vocabulary
2020-10-26 22:41:47,898: INFO : effective_min_count=40 retains 8160 unique words (11% of original 74065, drops 65905)
2020-10-26 22:41:47,899: INFO : effective_min_count=40 leaves 2627273 word corpus (87% of original 2988089, drops 360816)
2020-10-26 22:41:47,916: INFO : deleting the raw counts dictionary of 74065 items
2020-10-26 22:41:47,918: INFO : sample=0.001 downsamples 30 most-common words
2020-10-26 22:41:47,919: INFO : dow

In [11]:
model_name = "300fratures_40minwords_10context"
model.save(model_name)

2020-10-26 22:42:56,318: INFO : saving Word2Vec object under 300fratures_40minwords_10context, separately None
2020-10-26 22:42:56,319: INFO : not storing attribute vectors_norm
2020-10-26 22:42:56,320: INFO : not storing attribute cum_table
2020-10-26 22:42:56,461: INFO : saved 300fratures_40minwords_10context


In [13]:
def get_features(words, model, num_features):
    feature_vector = np.zeros((num_features),dtype=np.float32)

    num_words = 0
    index2word_set = set(model.wv.index2word)

    for w in words:
        if w in index2word_set:
            num_words += 1
            feature_vector = np.add(feature_vector, model[w])

    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [14]:
def get_dataset(reviews, model, num_features):
    dataset = list()

    for s in reviews:
        dataset.append(get_features(s, model, num_features))

    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

In [15]:
test_data_vecs = get_dataset(sentences, model, num_features)

c:\users\bitcamp\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [16]:
from sklearn.model_selection import train_test_split

In [18]:
x = test_data_vecs
y = np.array(sentiments)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42) 


In [19]:
from sklearn.linear_model import LogisticRegression

In [26]:
lgs = LogisticRegression(class_weight = 'balanced')
lgs.fit(x_train, y_train)

print("Acc: %f" %lgs.score(x_test, y_test))

Acc: 0.864600


c:\users\bitcamp\anaconda3\envs\nlp\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [29]:
test_clean = 'test_clean.csv'

test_data = pd.read_csv(data_path + test_clean)

test_review = list(test_data['review'])


In [45]:
test_data.head()

,review,id
0,naturally film main themes mortality nostalgia...,"""12311_10"""
1,movie disaster within disaster film full great...,"""8348_2"""
2,movie kids saw tonight child loved one point k...,"""5828_4"""
3,afraid dark left impression several different ...,"""7186_2"""
4,accurate depiction small time mob life filmed ...,"""12128_7"""


In [53]:
test_sentences = []
for data in test_review:
    test_sentences.append(data.split())
    

In [56]:
print(test_sentences[:])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [57]:
test_data_vecs = get_dataset(test_sentences, model, num_features)

c:\users\bitcamp\anaconda3\envs\nlp\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [58]:
pred = lgs.predict(test_data_vecs)
print("pred",pred)

pred [1 0 1 ... 0 1 1]


In [59]:
ids = list(test_data['id'])

ans = pd.DataFrame({'id': ids, 'sentiment': pred})

In [63]:
op = 'C:/Users/bitcamp/Desktop/NLP/NLP/tensorflow-ml-nlp-tf2/4.TEXT_CLASSIFICATION/output/'
  
if not os.path.exists(op):
    os.makedirs(op)

ans.to_csv(op + 'lgs_w2v.csv', index=False, quoting=3)